# Numerical Inverse inverse of Bruggeman 223_02 using mpmath

The Laplace transform for problem 223_2 is according to Ed Veling (23-12-2024)

$$L\{\phi(r,p)\}=\frac{h}{p}\frac{K_{0}\left(\beta r\sqrt{p}\right)}{K_{0}\left(\beta R\sqrt{p}\right)} $$

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import mpmath as mpm
from scipy.special import k0 as K0
import math

# Graver Stehfest algorithm

$$f_g(t, M) = \frac {\ln 2}{t}\sum_{k=1}^{2M}\zeta_k\hat{f}\left(\frac{k \ln 2}{t}\right)$$

where

$$\zeta_k=(-1)^{M + k}\sum_{j=[(k+1)/2]}^{k \bigwedge M}\frac{j^{M+1}}{M!}
\left(M \over j\right)
\left(2j \over j\right)
\left(j \over {k-j}\right)
$$

\left(\begin{array}{c} M\\j
\end{array}\right)

In [15]:
mpm.mp.dps = 15
mpm.mp.pretty = True
fp = lambda p: 1/(p + 1) ** 2
ft = lambda t: t * mpm.exp(-t)
tt = [0.001, 0.01, 0.1, 1., 10.]
ft(tt[0]), ft(tt[0]) - mpm.invertlaplace(fp, tt[0], method='talbot')

(0.000999000499833375, 8.57923043561212e-20)

In [14]:
# Stehfest

def n_over_k(n, k):
    return math.factorial(n) / (math.factorial(k) * (math.factorial(n-k)))

def zeta(k, M):        
        F = 0
        fac_M = math.factorial(M)
        for j in range(int((k + 1) / 2), min(k, M)):
            F += j ** (M + 1) / fac_M * n_over_k(M, j) * n_over_k(2 * j, j) * n_over_k(j, k - j)
        return (-1) ** (M + k) * F
    
def zetas(M):
    z = np.zeros(M, dtype=int)
    for k in range(M):
        z[k] = zeta(k, M)
    return z

    
def fhat(p, args):
    h, r, R, beta = args
    return h / p * K0(beta * r * np.sqrt(p)) / K0(beta * R * np.sqrt(p))

def F(fhat, t, M, args):
    """Stehfest back transformation."""
    zeta = zetas(M)
    t = np.array([t])
    s = np.zeros_like(t)
    for it, t_ in enumerate(t):
        for k in range(1, 2 * M + 1):
            s[it] += zeta[k] * fhat(k * np.log(2) / t_, *args)
        s[it] *= np.log(2) / t_
    return s
    
if __name__ == '__main__':
    times = np.logspace(-2, 2, 41)
    M = 10
    h, r, R, beta = 1., 30., 30., np.sqrt(0.2 / 600)
    Q= F(fhat, times, M, (h, r, R, beta))
        
    
   

TypeError: fhat() takes 2 positional arguments but 5 were given

In [9]:
mpm.mp.dps = 15
mpm.mp.pretty = True
def fp2(p, method='talbot'):
    global h, r,  R, beta
    return h / p * K0(beta * r * np.sqrt(p)) / K0(beta * R * np.sqrt(p))

h, r, R, beta = 1., 30., 30., 0.2/600.

tt = [0.01, 0.1, 1.0, 10., 100.]
mpm.invertlaplace(fp2, tt[0], mthod='talbot')

TypeError: ufunc 'k0' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [7]:
from inspect import signature
help(mpm.invertlaplace)

Help on method invertlaplace in module mpmath.calculus.inverselaplace:

invertlaplace(f, t, **kwargs) method of mpmath.ctx_mp.MPContext instance
    Computes the numerical inverse Laplace transform for a
    Laplace-space function at a given time.  The function being
    evaluated is assumed to be a real-valued function of time.

    The user must supply a Laplace-space function `\bar{f}(p)`,
    and a desired time at which to estimate the time-domain
    solution `f(t)`.

    A few basic examples of Laplace-space functions with known
    inverses (see references [1,2]) :

    .. math ::

        \mathcal{L}\left\lbrace f(t) \right\rbrace=\bar{f}(p)

    .. math ::

        \mathcal{L}^{-1}\left\lbrace \bar{f}(p) \right\rbrace = f(t)

    .. math ::

        \bar{f}(p) = \frac{1}{(p+1)^2}

    .. math ::

        f(t) = t e^{-t}

    >>> from mpmath import *
    >>> mp.dps = 15; mp.pretty = True
    >>> tt = [0.001, 0.01, 0.1, 1, 10]
    >>> fp = lambda p: 1/(p+1)**2
    >>> ft = lambd